In [ ]:
#5
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
import os
from PIL import Image

In [ ]:
africa_data = pd.read_csv('atlas_africa.csv', low_memory=False)

In [ ]:

africa_data = africa_data[africa_data['infection type'].notna()]

species_list = africa_data['Species'].unique()

output_dir = 'species_heatmaps'
os.makedirs(output_dir, exist_ok=True)

for species in species_list:
    fig, ax = plt.subplots(figsize=(10, 8))

    species_data = africa_data[africa_data['Species'] == species]

    total_isolates = species_data.groupby(['Year', 'antibiotics_class']).size()

    resistant_isolates = species_data[species_data['Status'] == 'Resistant'].groupby(['Year', 'antibiotics_class']).size()

    resistance_proportion = (resistant_isolates / total_isolates).unstack()

    resistance_proportion = resistance_proportion.fillna(0)

    sns.heatmap(resistance_proportion, cmap="coolwarm", ax=ax, 
                cbar_kws={'label': 'Proportion of Resistant Isolates'}, vmin=0, vmax=1)

    ax.set_title(f'{species} Resistance Over Time')

    ax.set_xlabel('Antibiotics Class')
    ax.set_ylabel('Year')
    ax.tick_params(axis='x', rotation=90)
    ax.tick_params(axis='y', rotation=0)

    plt.tight_layout()
    plt.savefig(os.path.join(output_dir, f'{species}_heatmap.png'))
    plt.close()

images = []
for species in species_list:
    image_path = os.path.join(output_dir, f'{species}_heatmap.png')
    images.append(Image.open(image_path))

gif_path = 'species_heatmaps.gif'
images[0].save(gif_path, save_all=True, append_images=images[1:], duration=3000, loop=0)